# About

This notebook parses the player selections into a tidy format

In [1]:
import pandas as pd

In [2]:
import re

In [3]:
# Reading in the raw data
selections_raw = pd.read_csv('data/player_selections.csv',index_col=0)

In [4]:
# Compiling the regex
regex = re.compile('\s(C|C|W|UTIL|D|G|CPT|FLEX)\s')
startRegex = re.compile('^(C|C|W|UTIL|D|G|CPT|FLEX)\s')

In [ ]:
# Creating a list to hold the raw data
parsedsSelections = []

for index, values in selections_raw.iterrows():
    selections = {}
    selections['contestID'] = values['contestID']
    selections['entryID'] = values['entryID']
    selections['entryName'] = values['entryName']
    
    selectionString = values['lineup']
    
    selectionString = selections_raw.iloc[1613]['lineup']
    
#     print(index, selectionString)
#     print('**********************************')
    
    matchIndexes = [match_span.span() for match_span in regex.finditer(selectionString)]
    
    selection = []
    
    firstMatchIndexes = startRegex.match(selectionString).span()
           
    # First Player  
    selection.append({'position':selectionString[0:firstMatchIndexes[1]-1], 
                      'player':selectionString[firstMatchIndexes[1]:matchIndexes[0][0]]})
    
    for i in range(0, len(matchIndexes)):
        if i < len(matchIndexes) - 1:
            # Player 2 to n-1
            
            selection.append(
                {
                    'position':selectionString[matchIndexes[i][0]+1:matchIndexes[i][1]-1], 
                    'player':selectionString[matchIndexes[i][1]:matchIndexes[i+1][0]]
                }
            )
        else:
            # Player n
            
            selection.append(
                {
                    'position':selectionString[matchIndexes[i][0]+1:matchIndexes[i][1]-1], 
                    'player':selectionString[matchIndexes[i][1]:]
                }
            )
                
    selections['selection'] = selection
    
    parsedsSelections.append(selections)
            
#     break

parsedsSelectionsDF = pd.DataFrame(parsedsSelections)

In [ ]:
# Exploding the selections into individual rows
parsedsSelectionsDF = parsedsSelectionsDF.explode('selection')

In [ ]:
# Expanding the selection column into 2 columns
parsedsSelectionsDF = pd.concat(
    [
        parsedsSelectionsDF.reset_index(drop=True),
        pd.json_normalize(parsedsSelectionsDF['selection'])
    ],
    axis=1)

In [ ]:
# Dropping unneccessary column
playerSelections = parsedsSelectionsDF.drop('selection',axis=1)

In [ ]:
playerSelections.to_csv('transformed_data/playerSelections.csv')